# Automatic Centerlines Extraction


First, extract the coronary artery centerlines from your dataset using the method described in 
[Coronary artery centerline extraction in cardiac CT angiography using a CNN-based orientation classifier (Walternik et al., 2019)](https://www.sciencedirect.com/science/article/pii/S1361841518308491). For now, CCTA scans from `<dataset_dir>` are matched with the patterns `*.mhd`
and `*_image.nii.gz`. 

```bash
source scripts/centerlines_autoextract.sh <dataset_dir> <output_dir>
```

For every `datapoint_X` in `<dataset_dir>`, this command creates files `<output_dir>/<datapoint_X>/vessel0.txt`,
`<output_dir>/<datapoint_X>/vessel1.txt` etc., where each `vessel<N>.txt` file corresponds to a discovered coranary 
artery vessel and it contains an array of 4D points (3D XYZ coordinates + artery lumen radius estimate).

## Create the HD5-encoded Dataset

The HD5 file type allows on-the-fly loading of big datasets, without the need to load entire arrays in memory -- 
relevant parts are loaded as needed, reducing memory footprint. 

For more inforamation, refer to the 
[HD5 Python documentation](https://docs.h5py.org/en/stable/).

In [ ]:
%cd ../

/home/marco/contrast-gan-3D


Adjust the variables in the following cell to match your setup:

In [ ]:
from pathlib import Path

DATASET_DIR = Path("/home/marco/data/MMWHS/ct_test")

OUTPUT_DIR = DATASET_DIR / "auto_centerlines"

# By default, .h5 converted files are saved inside `datset_dir`; set this
# variable if you want to save them somewhere else
H5_OUTPUT_DIR = None

print(str(DATASET_DIR))
print(str(OUTPUT_DIR))
print(str(H5_OUTPUT_DIR))

/home/marco/data/MMWHS/ct_test
/home/marco/data/MMWHS/ct_test/auto_centerlines
None


In [ ]:
from pprint import pprint

cctas = sorted(
    list(DATASET_DIR.glob("*.mhd")) + list(DATASET_DIR.glob("*_image.nii.gz"))
)
centerlines = sorted([d for d in OUTPUT_DIR.glob("*") if d.is_dir()])
print(f"Found {len(cctas)} CCTAs with {len(centerlines)} centerlines")

zipped = list(zip(cctas, centerlines))
pprint(zipped[:3])

Found 40 CCTAs with 40 centerlines
[(PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2001_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2001_image.nii.gz')),
 (PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2002_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2002_image.nii.gz')),
 (PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2003_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2003_image.nii.gz'))]


In [ ]:
from contrast_gan_3D.utils import io_utils, logging_utils

logging_utils.set_project_loggers_level(level="DEBUG")

contrast_gan_3D.utils.io_utils: INFO -> DEBUG


In [ ]:
for ccta, centerlines_dir in zipped:
    if str(centerlines_dir).split("/")[-1] == str(ccta).split("/")[-1]:
        io_utils.sitk_to_h5(ccta, centerlines_dir, h5_output_dir=H5_OUTPUT_DIR)
    else:
        print(f"**No centerlines for {str(ccta)!r}**")

[2024-01-12 15:03:32,228: DEBUG] Saved centerlines to '/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2001_image.nii.gz/centerlines.npz' with key 'centerlines' (contrast_gan_3D.utils.io_utils:61)
[2024-01-12 15:03:32,228: DEBUG] CCTA: (512, 512, 224) centerlines: (756, 4) (contrast_gan_3D.utils.io_utils:86)
[2024-01-12 15:03:32,229: DEBUG] H5 file: '/home/roel/data/MMWHS/ct_test/ct_test_2001_image.h5' (contrast_gan_3D.utils.io_utils:91)
[2024-01-12 15:03:34,619: DEBUG] Saved centerlines to '/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2002_image.nii.gz/centerlines.npz' with key 'centerlines' (contrast_gan_3D.utils.io_utils:61)
[2024-01-12 15:03:34,621: DEBUG] CCTA: (512, 512, 200) centerlines: (10132, 4) (contrast_gan_3D.utils.io_utils:86)
[2024-01-12 15:03:34,622: DEBUG] H5 file: '/home/roel/data/MMWHS/ct_test/ct_test_2002_image.h5' (contrast_gan_3D.utils.io_utils:91)
[2024-01-12 15:03:37,205: DEBUG] Saved centerlines to '/home/marco/data/MMWHS/ct_test/auto_centerl